In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import sys
sys.version

'3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]'

# Importing modules
The following commands should execute with no output

In [23]:
import glob
import mne
from mne.preprocessing import ICA, EOGRegression
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [24]:
from preprocessing import preprocessing, readEpochDenoise
from patient_data import patient_data
from Hjorths import hjorths, getFeaturesIntoDataframe
from ML import ML, get_ML_Results, highlight_max_clr, highlight_max_acc, plotCM
from DL import EEGNet

## Pre-processing
### Splitting the features of Raw EEG data and assigning them to their correct subject group.

In [25]:
data_path = glob.glob('Raw/*.set')
DLB      = [i for i in data_path if 'DLB' in i.split('\\')[1]]
AD     = [i for i in data_path if 'F001' in i.split('\\')[1]]
PDD     = [i for i in data_path if 'F023' in i.split('\\')[1]]
PD      = [i for i in data_path if 'G20' in i.split('\\')[1]]
HC = [i for i in data_path if 'Kontroll' in i.split('\\')[1]]

print(len(DLB), len(AD), len(PDD), len(PD), len(HC))

16 19 13 15 21


### In this section the RAW EEG data for each subject group have been read, pre-processed and saved into a file.

In [26]:
# read, clean, and epoch all the data
if False:
    cleaned_data = readEpochDenoise(DLB, AD, PDD, PD, HC,  apply_wavelet = False, apply_notch = False)

## Load preprocessed data

In [27]:
groups = ['DLB', 'AD', 'PDD', 'PD', 'HC']
events = ['Standard', 'Target', 'Distractor', ]

In [28]:
def pkl_load(pp_file):
    with open(pp_file, 'rb') as handle:
        v_out = np.array(pickle.load(handle), dtype=object)
    return v_out


In [29]:
# Standard epochs ica-cleaned
ppdir = ''
DLB_Standard  = pkl_load(ppdir+'preprocessed_data/Standard_DLB.pkl')
AD_Standard   = pkl_load(ppdir+'preprocessed_data/Standard_AD.pkl')
PDD_Standard  = pkl_load(ppdir+'preprocessed_data/Standard_PDD.pkl')
PD_Standard   = pkl_load(ppdir+'preprocessed_data/Standard_PD.pkl')
HC_Standard   = pkl_load(ppdir+'preprocessed_data/Standard_HC.pkl')
print(np.array(DLB_Standard).shape , np.array(AD_Standard).shape , np.array(PDD_Standard).shape, np.array(PD_Standard).shape, np.array(HC_Standard).shape)

(16, 300, 9, 205) (19, 300, 9, 205) (13, 300, 9, 205) (15, 300, 9, 205) (21, 300, 9, 205)


In [30]:
# target epochs 
ppdir = ''
DLB_Target  = pkl_load(ppdir+'preprocessed_data/Target_DLB.pkl')
AD_Target   = pkl_load(ppdir+'preprocessed_data/Target_AD.pkl')
PDD_Target  = pkl_load(ppdir+'preprocessed_data/Target_PDD.pkl')
PD_Target   = pkl_load(ppdir+'preprocessed_data/Target_PD.pkl')
HC_Target   = pkl_load(ppdir+'preprocessed_data/Target_HC.pkl')
print(DLB_Target.shape, AD_Target.shape, PDD_Target.shape, PD_Target.shape, HC_Target.shape)

(16,) (19,) (13,) (15,) (21, 75, 9, 205)


In [31]:
# distractor epochs
ppdir = ''
DLB_Distractor  = pkl_load(ppdir+'preprocessed_data/Distractor_DLB.pkl')
AD_Distractor   = pkl_load(ppdir+'preprocessed_data/Distractor_AD.pkl')
PDD_Distractor  = pkl_load(ppdir+'preprocessed_data/Distractor_PDD.pkl')
PD_Distractor   = pkl_load(ppdir+'preprocessed_data/Distractor_PD.pkl')
HC_Distractor   = pkl_load(ppdir+'preprocessed_data/Distractor_HC.pkl')
print(DLB_Distractor.shape, AD_Distractor.shape, PDD_Distractor.shape, PD_Distractor.shape, HC_Distractor.shape)

(16,) (19,) (13,) (15,) (21, 75, 9, 205)


## Compute Hjorth's parameters

In [32]:
# compute Hjorth's parameters for standard epochs
DLB_HP_Standard = hjorths(DLB_Standard, average = False) # DLB
AD_HP_Standard  = hjorths(AD_Standard, average = False) # AD
PDD_HP_Standard = hjorths(PDD_Standard, average = False) # PD
PD_HP_Standard  = hjorths(PD_Standard, average = False) # PDD
HC_HP_Standard  = hjorths(HC_Standard, average = False) # HC

In [33]:
# compute Hjorth's parameters for target epochs
DLB_HP_Target = hjorths(DLB_Target, average = False) # DLB
AD_HP_Target  = hjorths(AD_Target, average = False) # AD
PDD_HP_Target = hjorths(PDD_Target, average = False) # PD
PD_HP_Target  = hjorths(PD_Target, average = False) # PDD
HC_HP_Target  = hjorths(HC_Target, average = False) # HC

In [34]:
# compute Hjorth's parameters for distractor epochs
DLB_HP_Distractor = hjorths(DLB_Distractor, average = False) # DLB 
AD_HP_Distractor  = hjorths(AD_Distractor, average = False)  # AD
PDD_HP_Distractor = hjorths(PDD_Distractor, average = False) # PD 
PD_HP_Distractor  = hjorths(PD_Distractor, average = False)  # PDD
HC_HP_Distractor  = hjorths(HC_Distractor, average = False) # HC

In [80]:
gi = 0
ei = 0
pi = 0 # patient
g = groups[gi]
e = events[pi]

#print(f'{g}_HP_{e}.hjorth_parameters')
xh = eval(f'{g}_HP_{e}.hjorth_parameters')
len(xh)
x = np.array(xh[pi])
Nh, Ne, Nc = x.shape
p = np.array([gi, ei, pi])
p = np.tile(p,(Ne,1))
np.hstack((p, x[0], x[1], x[2])).shape


(300, 30)

In [88]:

col = ['group', 'event', 'patient']
hc = ['H'+str(i) for i in range(3)]
cc = ['C'+str(i) for i in range(9)]
print(hc)
print(cc)
    
HC = [h+'_'+c for h in hc for c in cc]
print(HC)
cols = col + HC
cols


['H0', 'H1', 'H2']
['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8']
['H0_C0', 'H0_C1', 'H0_C2', 'H0_C3', 'H0_C4', 'H0_C5', 'H0_C6', 'H0_C7', 'H0_C8', 'H1_C0', 'H1_C1', 'H1_C2', 'H1_C3', 'H1_C4', 'H1_C5', 'H1_C6', 'H1_C7', 'H1_C8', 'H2_C0', 'H2_C1', 'H2_C2', 'H2_C3', 'H2_C4', 'H2_C5', 'H2_C6', 'H2_C7', 'H2_C8']


['group',
 'event',
 'patient',
 'H0_C0',
 'H0_C1',
 'H0_C2',
 'H0_C3',
 'H0_C4',
 'H0_C5',
 'H0_C6',
 'H0_C7',
 'H0_C8',
 'H1_C0',
 'H1_C1',
 'H1_C2',
 'H1_C3',
 'H1_C4',
 'H1_C5',
 'H1_C6',
 'H1_C7',
 'H1_C8',
 'H2_C0',
 'H2_C1',
 'H2_C2',
 'H2_C3',
 'H2_C4',
 'H2_C5',
 'H2_C6',
 'H2_C7',
 'H2_C8']

In [121]:
for gi, g in enumerate(groups):
    for ei, e in enumerate(events):
        
        #print(f'{g}_HP_{e}')
        xh = eval(f'{g}_HP_{e}.hjorth_parameters')
        Np = len(xh)
        
        for pi in range(Np):
            x = np.array(xh[pi])
            Nh, Ne, Nc = x.shape # parameters epochs, channels
            #print(Ne)
            p = np.array([gi, ei, pi])
            #print(p)
            P = np.array(np.tile(p,(Ne,1)))
            #print(P)
            t = np.hstack((P, x[0], x[1], x[2]))
            if pi == 0:    
                print(f'{g} {e} Np =  {Np}, Nh =  {Nh}, Ne = {Ne}, Nc = {Nc}')
            if gi == 0 and ei == 0 and pi == 0:
                d = t
            else:
                d = np.append(d, t, axis = 0)
            print(f'{t.shape}')




                

DLB Standard Np =  16, Nh =  3, Ne = 300, Nc = 9
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
DLB Target Np =  16, Nh =  3, Ne = 75, Nc = 9
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(60, 30)
(75, 30)
DLB Distractor Np =  16, Nh =  3, Ne = 75, Nc = 9
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(57, 30)
(75, 30)
AD Standard Np =  19, Nh =  3, Ne = 300, Nc = 9
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
(300, 30)
AD Target Np =  19, Nh =  3, Ne = 75, Nc = 9
(75, 30)
(33, 30)
(75, 30)
(61, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30

(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
HC Distractor Np =  21, Nh =  3, Ne = 75, Nc = 9
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)
(75, 30)


In [129]:
df = pd.DataFrame(d,columns=cols)
df = df.astype({'group': int, 'event': int, 'patient': int})

In [132]:
print(df)

       group  event  patient         H0_C0         H0_C1         H0_C2  \
0          0      0        0  4.264041e-11  2.451439e-11  3.548383e-11   
1          0      0        0  8.436027e-11  4.828491e-11  5.620093e-11   
2          0      0        0  1.000064e-10  4.462723e-11  1.026924e-10   
3          0      0        0  4.466278e-11  3.857130e-11  3.733267e-11   
4          0      0        0  4.937119e-11  5.190647e-11  3.956241e-11   
...      ...    ...      ...           ...           ...           ...   
37594      4      2       20  5.904289e-12  4.220885e-12  4.868758e-12   
37595      4      2       20  9.866072e-12  5.021750e-12  7.386896e-12   
37596      4      2       20  8.437395e-12  6.951848e-12  8.270025e-12   
37597      4      2       20  3.614260e-12  3.613806e-12  5.616614e-12   
37598      4      2       20  5.838954e-12  5.430470e-12  7.670546e-12   

              H0_C3         H0_C4         H0_C5         H0_C6  ...     H1_C8  \
0      6.760429e-11  5.379963e-

In [131]:
df.to_csv('df_hjorth.csv')